In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
from handler import geozoom_handler
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from attn_model2 import *
from fc_net import *
from helpers import *
from utils2 import *

In [2]:
# random.sample(range(0, len(image_names)), 5)

In [3]:
# image_names = get_png_names("../pooling/data/MEX2/")#[100:105]
image_names = get_png_names("../pooling/data/MEX2/")
image_indices = random.sample(range(0, len(image_names)), 20)
image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../pooling/data/migration_data.json" , image_names)

20 municipalities.


In [4]:
for i in image_names:
    print(load_inputs(i).shape)

torch.Size([1, 3, 1033, 1807])
torch.Size([1, 3, 278, 610])
torch.Size([1, 3, 1002, 872])
torch.Size([1, 3, 965, 668])
torch.Size([1, 3, 1251, 1823])
torch.Size([1, 3, 1033, 687])
torch.Size([1, 3, 125, 193])
torch.Size([1, 3, 1674, 2084])
torch.Size([1, 3, 260, 267])
torch.Size([1, 3, 242, 336])
torch.Size([1, 3, 560, 715])
torch.Size([1, 3, 756, 847])
torch.Size([1, 3, 977, 1185])
torch.Size([1, 3, 1629, 1589])
torch.Size([1, 3, 830, 1008])
torch.Size([1, 3, 2031, 2268])
torch.Size([1, 3, 458, 329])
torch.Size([1, 3, 976, 1418])
torch.Size([1, 3, 503, 459])
torch.Size([1, 3, 419, 493])


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNetBinary(in_channels = 512, h = 7, w = 7, batch_size = 1, resnet = resnet18).to(device)
lr = .0001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

fc_model = fc_net(resnet = resnet18).to(device)
fc_optimizer = torch.optim.Adam(fc_model.parameters(), lr = .01)

butler = geozoom_handler(attn_model, 
                         fc_model, device, 
                         criterion, 
                         attn_optimizer, 
                         fc_optimizer, 
                         convergence_dims = (358, 284),
                         plot = False, 
                         v = False)



In [6]:
BATCH_SIZE = 1
SPLIT = .70

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [7]:
for i,o in val_dl:
    print(load_inputs(i[0]).shape, o)

torch.Size([1, 3, 260, 267]) tensor([10.], dtype=torch.float64)
torch.Size([1, 3, 242, 336]) tensor([101.], dtype=torch.float64)
torch.Size([1, 3, 278, 610]) tensor([1364.], dtype=torch.float64)
torch.Size([1, 3, 976, 1418]) tensor([618.], dtype=torch.float64)
torch.Size([1, 3, 1629, 1589]) tensor([2526.], dtype=torch.float64)
torch.Size([1, 3, 458, 329]) tensor([8.], dtype=torch.float64)


In [8]:
butler.train_attn_model(train_dl, val_dl)

Epoch:  0
  Training Loss:  1387.1718320165362
  Validation Loss:  772.771844069163
Loss thresholds for training:  [0.0, 198.1674045737909, 396.3348091475818, 594.5022137213726, 792.6696182951636, 990.8370228689545, 1189.0044274427453]
Starting from threshold:  6  with value:  1189.0044274427453


Epoch:  1
  Training Loss:  1304.0575648716517
  Validation Loss:  791.1154737472534


Epoch:  2
  Training Loss:  1201.4100080217634
  Validation Loss:  799.9740470250448


Epoch:  3
  Training Loss:  1099.6350555419922
  Validation Loss:  790.6566088994344
  Moving to threshold:  5   |  Next loss benchmark:  990.8370228689545


/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")




Epoch:  4
  Training Loss:  1289.846396309989
  Validation Loss:  932.1488068898519


Epoch:  5
  Training Loss:  1193.7293069022041
  Validation Loss:  2504.8285643259683


Epoch:  6
  Training Loss:  1112.9698093959264
  Validation Loss:  3798.4180183410645


Epoch:  7
  Training Loss:  1036.013643537249
  Validation Loss:  2315.063376108805


Epoch:  8
  Training Loss:  897.5735361916678
  Validation Loss:  6023.48117462794
  Moving to threshold:  4   |  Next loss benchmark:  792.6696182951636


Epoch:  9
  Training Loss:  975.5095773424421
  Validation Loss:  6023.48117462794


Epoch:  10
  Training Loss:  839.3008178983416
  Validation Loss:  6023.48117462794


Epoch:  11
  Training Loss:  905.5897182737078
  Validation Loss:  6023.48117462794


Epoch:  12
  Training Loss:  776.2665696144104
  Validation Loss:  6023.48117462794
  Moving to threshold:  3   |  Next loss benchmark:  594.5022137213726


Epoch:  13
  Training Loss:  1047.9971351623535
  Validation Loss:  6023.4811746

KeyboardInterrupt: 

In [ ]:
butler.image_sizes

In [ ]:
butler.threshold_weights.keys()

In [ ]:
for i,o in train_dl:
    print(load_inputs(i[0]).shape)

In [ ]:
641 * .70

In [ ]:
263 - 

In [8]:
for i, o in train_dl:
    
    muni_id = i[0].split("/")[4]
    
    plt.imshow(butler.prep_input(i).detach().cpu()[0].permute(1,2,0))
    plt.title(butler.image_sizes[muni_id])
    plt.savefig(f"test{muni_id}.png")
    
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [9]:
# # for i in range(50):
# while handler.threshold_index > 0:

#     for impath, output in train_dl:

#         # Prep the input and pass it to the trainer (this could easily be done in one step eventually if ya want)
#         input = handler.prep_input(impath)
#         handler.train(input, output)
    
#     handler.end_epoch(train_dl, val_dl = None)

In [8]:
handler.image_sizes

{'484002003': (0, 224, 0, 224),
 '484016045': (0, 224, 0, 224),
 '484020489': (0, 224, 0, 224),
 '484020254': (0, 224, 0, 224)}

In [11]:
list(butler.threshold_weights.keys())

[6]

In [2]:
# for i, o in val_dl:
#     print(i, o)

In [1]:
# def run_validation(attn_model, fc_model, input, weights_dict, plot = False):
    
#     muni_id = input[0].split("/")[4]
#     cur_image = load_inputs(input[0])    
    
#     for k in weights_dict.keys():
                
#         if k != 'fc':
            
#             model = attn_model
#             model.load_state_dict(weights_dict[k])
#             model.eval()
#             IM_SIZE = (cur_image.shape[2], cur_image.shape[3])
#             gradcam, attn_heatmap = get_gradcam(model, IM_SIZE, cur_image.cuda()) 
#             cur_image, new_dims = butler.clip_input(cur_image, attn_heatmap)
            
#             if plot == True:
            
#                 plot_gradcam(gradcam)
#                 plt.savefig(f"threshold{k}_muni{muni_id}.png")
#                 plt.clf()
            
        
#         if k == 'fc':
            
#             model = fc_model
#             model.load_state_dict(weights_dict[k])
#             model.eval()
#             return model(cur_image.cuda()).item()
            

        
# for i, o in train_dl:

#     print(round(run_validation(attn_model, fc_model, i, butler.threshold_weights), 2), "     ", o.item())
    